# Pre requisites

If you were running this on your own machine you would have to install jupyter, neuroconv package and the data.

However, if you are using either binder or github codespaces for the workshop these steps have already been done for you so you don't need to install anything.


1- Download, install and open GUIDE : https://nwb-guide.readthedocs.io/en/stable/

2- Get datasets on your local machine : 

Download 2 NWB datasets on Pasteur Drive (Total 23MB)

https://drive.pasteur.fr/s/ipLGkdP83Mwo975 

with the following password: WorkshopNWB@2025


# NWB workshop

## PART 1 : Reading & Exploring a NWB file 

### Dataset 1 : Behavioral data (Go/NoGo auditory task)

In this section, you will load NWB datasets with the no-code tool "GUIDE".

Here, we explore a dataset acquired with BASIL app which is used to train mice to operant conditionning auditory discrimination task. One sound (Go) is associated to a reward and another sound (Nogo) is associated to no reward / punishment.

1- In the GUIDE app, click Explore, load and find in your local repository the file sub-3502_ses-13.nwb

3502 is the subject (mouse) ID.

13 is the session (acquisition) ID.

2- Expand the Neurodata Tab

3- Check the « Lick » and click the blue button on top. Zoom to 20s of experiments. 

4- Close and now check these 2 items : Lick, Reward. Click the blue button on top. Go a bit after the beginning.  
Explanation on how to interpret these data are documented in the "description" part.

5- Close. Now, in Matlab or Python, with few line of codes, you can open and visualize the data in a notebook as follow : 

### Look at the data structure and visualize behavior data

In [107]:
# import libraries
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import groupby

In [ ]:
folder_path = "C:/Users/cdussaux/Documents/Python/NWB_conversion/data/sub-3502_ses-13.nwb"
io = NWBHDF5IO(folder_path, mode='r')
nwbfile = io.read()
nwbfile

In [109]:
# Load Reward data 
rewarddata = nwbfile.acquisition['Reward'].data
Bin = 1./nwbfile.acquisition['Reward'].rate
tps_trial = np.arange(1, len(rewarddata) + 1) * Bin

# Load trial type data 
trialid = nwbfile.acquisition['TrialType'].data

# Load Lick data data
Lickdata = nwbfile.acquisition['Lick'].data
Binlick = 1./nwbfile.acquisition['Lick'].rate
tps_data = np.arange(1, len(Lickdata) + 1) * Binlick

In [ ]:
# Visualize timeseries
plt.plot(tps_data[:], Lickdata[:])
plt.plot(tps_trial[:], trialid[:])
plt.plot(tps_trial[:], rewarddata[:], "r-")

plt.ylim([0, 10])
plt.xlim([300, 320])

In [111]:
# Load whichsound.bin - in an ulterior version, it will be included in the NWB. 
def read_whichsound_bin(fpath):
    file_path = fpath + 'WhichSound.bin'
    dataid = np.fromfile(file_path, dtype='float64')
    return dataid

fpath = 'U:/Bathellierlab_gaia/BASIL/BASIL_FAIR/BASILapp/Results/TGTD/NAJ/Ribeye project/Ctrl vs RIb ll inj bilat (deaf)/M3502/20250909/123716_Data/'
dataid = read_whichsound_bin(fpath)

In [ ]:
def process_sequence_simple_auto(arr):
    """
    Fonction pour extraire la liste des sons, en incluant les répétitions du fichier whichsound.
    """
    if len(arr) == 0:
        return np.array([])
    
    # Trouver la longueur minimale des séquences de son
    lengths = []
    for key, group in groupby(arr):
        length = len(list(group))
        lengths.append(length)
    
    # Prendre la longueur minimale (mais au moins 1)
    repetition_length = min(lengths) if lengths else 1
    
    #print(f"Longueur de répétition détectée: {repetition_length}")
    
    result = []
    for key, group in groupby(arr):
        group_list = list(group)
        num_occurrences = len(group_list) // repetition_length
        result.extend([key] * num_occurrences)
    
    return np.array(result)

# Extraire la séquence unique des sons de whichsound
AllTrials = process_sequence_simple_auto(dataid)
print(AllTrials[:20])

In [ ]:
# Trouver le nombre de sons uniques (en ignorant les NaN)
valid_trials = AllTrials[~np.isnan(AllTrials)]
AllTrials = AllTrials[~np.isnan(AllTrials)]
if len(valid_trials) > 0:
    SoundNum = int(np.nanmax(np.unique(valid_trials)))
else:
    SoundNum = 0

print(f"Nombre de sons: {SoundNum}")

In [114]:
# On construit le vecteur Correct qui permet de quantifier les performances de l'animal au cours de la session
trialid = np.array(trialid)
trial_start_id = np.where(trialid == 9)[0]
sound_start_id = np.where(trialid == 99)[0]

# Create Correct_out with the appropriate length
Correct_out = np.zeros(len(sound_start_id), dtype=int)

for k in range(len(sound_start_id)):
    if k == len(sound_start_id) - 1:
        respwin = trialid[trial_start_id[k]+1:]
        rewardid = rewarddata[trial_start_id[k]+1:]
    else:
        # Regular case
        respwin = trialid[trial_start_id[k]+1 : trial_start_id[k+1]-1]
        rewardid = rewarddata[trial_start_id[k]+1 : trial_start_id[k+1]-1]
    
    # Supprimer les valeurs 99 qui correspondent au début du son
    respwin = respwin[respwin != 99]
    
    # Trouver les valeurs uniques
    outcometrial = np.unique(respwin)
    
    try:
        rewardout = rewardid[rewardid > 0]
    except:
        rewardout = np.array([0])
    
    # Supprimer les valeurs 4 qui correspondent à des reward manuel
    rewardout = rewardout[rewardout != 4]
    
    if len(rewardout) == 0:
        rewardout = np.array([0])
    
    # Déterminer si le trial est correct
    if (3 in rewardout) or (1 in rewardout):
        Correct_out[k] = 1
    elif (0 in rewardout) and (2 in outcometrial):
        Correct_out[k] = 1
    else:
        Correct_out[k] = 0

In [ ]:
Correct = np.array(Correct_out)
Perf = np.nanmean(Correct)

# Calcul des proportions
if SoundNum > 0:
    PropCorrect = np.full(SoundNum, np.nan)
    for sd in range(1, SoundNum + 1):
        indices = (AllTrials == sd)
        if np.any(indices):
            PropCorrect[sd-1] = np.nanmean(Correct[indices])
    
    # Inverser les proportions pour la première moitié
    PropCorrect[0:SoundNum//2] = 1 - PropCorrect[0:SoundNum//2]
    
    # Graphique
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, SoundNum + 1), PropCorrect, '.-', linewidth=3, markersize=20)
    plt.ylim([0, 1])
    plt.xlabel('Sound Identity')
    plt.ylabel('High proba')
    plt.axvline(x=SoundNum/2 + 0.5, color='k', linestyle='-')
    plt.title(f'Performance: {Perf:.3f}')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Aucune donnée à analyser")

In [116]:
io.close()

### Dataset 2 : Electrophysiological recordings + Sound stimuli

Here, we explore a dataset including : 

- Preprocessing / Spike sorting of Neuropixel recordings acquired with OpenEphys. Preprocessing is done with Kilosort and data curation is performed with Phy tool.  

- Behavior/stimuli data acquired with BASIL.

- EVENT TTL from BASIL to ephys for offline data synchronization.

1- In the GUIDE app, click Explore, load and find in your local repository the file sub-716_ses-27_res.nwb

716 is the subject (mouse) ID.

27 is the session (acquisition) ID.

Dataset courtesy : Pierre Platel, DSAPM.

2- Expand the Neurodata Tab

3- Expand the « acquisition » Tab. 

4- Check the « Lick » and click the blue button on top. Zoom to 20s of experiments. 

5- Close and now check 2 of these items : Lick, TTLTrignpx. Click the blue button on top. Zoom on 50s. 

6- Close. With few line of codes, you can open and visualize the data in a notebook as follow : 

### Look at the data structure and visualize behavior data

In [ ]:
folder_path = "C:/Users/cdussaux/Documents/Python/NWB_conversion/data/sub-716_ses-27_res.nwb"
io = NWBHDF5IO(folder_path, mode='r')
nwbfile = io.read()
nwbfile

In [ ]:
# Visualize timeseries
plt.plot(nwbfile.acquisition['Lick'].data[100:20000])
plt.plot(nwbfile.acquisition['TTLtrignpx'].data[100:20000])

7- Go back to GUIDE app, expand the «unit» Tab and have a look at the units description. 

8- Click the "Raster" button in the "units" checkbox.

Select > 100 units with the "+" 

Slide on 10s timewindows.

9- Close and go to Python usage. With few line of codes, you can open and visualize the data in a notebook as follow :


### Look at the data structure and plot spike raster plot

In [119]:
import numpy as np

units = nwbfile.units
units_spike_times = units["spike_times"]
# bin size for counting spikes
time_resolution = 0.01

# start and end times (relative to the stimulus at 0 seconds) that we want to examine and align spikes to
window_start_time = -0.1
window_end_time = 2

# time bins used
n_bins = int((window_end_time - window_start_time) / time_resolution)
bin_edges = np.linspace(window_start_time, window_end_time, n_bins, endpoint=True)

# useful throughout analysis
n_units = len(units_spike_times)

In [ ]:
n_trials = 1

# 3D spike matrix to be populated with spike counts
spike_matrix = np.zeros((n_units, len(bin_edges), n_trials))

# populate 3D spike matrix for each unit for each stimulus trial by counting spikes into bins
for unit_idx in range(n_units):
    spike_times = units_spike_times[unit_idx]
    
    # get spike times that fall within the bin's time range relative to the stim time        
    first_bin_time = bin_edges[0]
    last_bin_time = bin_edges[-1]
    first_spike_in_range, last_spike_in_range = np.searchsorted(spike_times, [first_bin_time, last_bin_time])
    spike_times_in_range = spike_times[first_spike_in_range:last_spike_in_range]

    # convert spike times into relative time bin indices
    bin_indices = ((spike_times_in_range - (first_bin_time)) / time_resolution).astype(int)
    
    # mark that there is a spike at these bin times for this unit on this stim trial
    for bin_idx in bin_indices:
        spike_matrix[unit_idx, bin_idx, 0] += 1

spike_matrix.shape

In [ ]:
trial = 0
fig, ax = plt.subplots(figsize=(10,10))

ax.set_title("Unit Spikes")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Unit #")

img = ax.imshow(spike_matrix[:,:,trial], extent=[window_start_time,window_end_time,0,n_units], aspect=0.001, vmin=0, vmax=1)
cbar = fig.colorbar(img, shrink=0.5)
cbar.set_label("# Spikes")

In [122]:
io.close()

## PART 2 : Building a multimodal NWB file using neuroconv

In this section, you will create the previous NWB dataset 2 using : 

1- NWB BASIL file

2- RAW ephys file (just for the example but too heavy)

3- RAW Kilosort + Phy file

4- EVENT TTL from BASIL to ephys for synchronisation

5- Behavior video file as external link

In [96]:
# import libraries
from pathlib import Path
from pynwb import NWBHDF5IO
import os
import shutil
import numpy as np
from pynwb import TimeSeries
from scipy.io import loadmat
import pandas as pd

In [ ]:
# Provide saving and data paths
saving_data_dir = Path("C:/Users/cdussaux/Documents/Python/NWB_conversion/data")

# Select BASIL NWB file 
basil_NWB_dir = Path("C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data")
basil_NWB_filename = "sub-716_ses-27.nwb" 
basil_NWBdata = basil_NWB_dir / basil_NWB_filename

# Select open ephys RAW data file (folder and stream) 
#folder_path_ephys = "U:/Bathellierlab_gaia/Data/Electrophysiology/Pierre/RAW_DATA/716_20250915_DelGNG_AC&M2_3&3/Record Node 112"
#stream_name = 'Record Node 112#Neuropix-PXI-116.ProbeB-AP'

# Select RAW preprocessed data (spike sorting)
folder_path_phy = "C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data/716_20250915_DelGNG_M2_3"

# Select event TTL sent by BASIL to open ephys 
stimTTLtmp = "C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data/stimTTL/timestamps.npy" #"/experiment1/recording1/events/Neuropix-PXI-116.ProbeA-AP/TTL/timestamps.npy"
stimTTL = stimTTLtmp #folder_path_ephys + stimTTLtmp

# Select NPX timestamp
timestamptmp = "C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data/timestamp/timestamps.npy" #"/experiment1/recording1/continuous/Neuropix-PXI-116.ProbeA-AP/timestamps.npy"
timestamp = timestamptmp # folder_path_ephys + timestamptmp

# Select Video file
raw_video_path = "C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data/video/"  #"U:/Bathellierlab_gaia/BASIL/BASIL_FAIR/BASILapp/Results/Pierre/M716/20250915/143731_Data/"
video_file_path = raw_video_path + "MouseVideo1.avi"
timing_files = raw_video_path + "VideoTiming1.mat"

# Add csv log file to NWB
csv_path = "C:/Users/cdussaux/Documents/Python/NWB_workshop/NWBworkshop/data/"
csv_file_path = csv_path + "Trial_log.csv"
csv_file_path_copy = csv_path + "Trial_log_copy.csv"

# name of the new file is duplicated in the saving_data_dir.
saving_data_filename, ext = os.path.splitext(basil_NWB_filename)
saving_data_filename = saving_data_filename + "_res" + ext
NWBdata_concatenate = saving_data_dir / saving_data_filename
NWBdata_concatenate

In [ ]:
# Make a copy of BASIL nwb file
shutil.copyfile(basil_NWBdata, NWBdata_concatenate)

In [ ]:
io1 =  NWBHDF5IO(NWBdata_concatenate, mode='r')
nwbfile = io1.read()
nwbfile

In [100]:
io1.close()

In [101]:
# Create Timeseries required in the NWB file. 

ttl_timestamps = np.load(stimTTL)
stimulusTTL = TimeSeries(
    name="TTL",
    data=ttl_timestamps,
    unit="a.u.",
    timestamps=ttl_timestamps
)

timestamp = np.load(timestamp)
stimulusTimestamp = TimeSeries(
    name="timestamp",
    data=timestamp,
    unit="a.u.",
    timestamps=timestamp
)

In [102]:
# CSV PART : Create a csv with the data structure allowing data incorportation in the NWB file. 
df = pd.read_csv(csv_file_path, sep=';')

# Ajouter artificiellement les colonnes 'start_time' et 'stop_time'
df['start_time'] = 0 # Fake data 
df['stop_time'] = 0 # Fake data 

# Placer les nouvelles colonnes en première position (optionnel, pour respecter l'ordre)
new_columns = ['start_time', 'stop_time'] + [col for col in df.columns if col not in ['start_time', 'stop_time']]
df = df[new_columns]

# Sauvegarder avec le nouveau header
df.to_csv(csv_file_path_copy, sep=',', index=False)

In [103]:
# VIDEO PART : Create video file as timeseries to be integrated in the NWB file.
data = loadmat(timing_files)
video_timestamps = data['VideoTiming'].squeeze()

timestamps_video = TimeSeries(
    name="timestamps_video",
    data=video_timestamps,
    unit="s",
    timestamps=video_timestamps
)

In [104]:
from neuroconv.datainterfaces import ExternalVideoInterface, PhySortingInterface, CsvTimeIntervalsInterface #OpenEphysRecordingInterface

# Create open ephys conversion interface 
# interface_openephys = OpenEphysRecordingInterface(folder_path=folder_path_ephys, stream_name=stream_name)

# create spike sorting interface 
interface_phy = PhySortingInterface(folder_path=folder_path_phy, verbose=False)

# create video behavior interface
interface_behavior = ExternalVideoInterface(file_paths=[video_file_path], verbose=False, video_name="BehaviorVideo")

# create csv interface
csv_interface = CsvTimeIntervalsInterface(file_path=csv_file_path_copy, verbose=False)

with NWBHDF5IO(basil_NWBdata, mode = 'r') as fin, NWBHDF5IO(NWBdata_concatenate, mode = 'w' ) as fout:
    datatmp = fin.read() # Read nwb file from BASIL
    interface_behavior.add_to_nwbfile(datatmp) # Add the behavior video
    datatmp.add_acquisition(timestamps_video) # Add the timestamps cam
    interface_phy.add_to_nwbfile(datatmp) # Add the phy information
    datatmp.add_acquisition(stimulusTTL) # Add the TTL event
    datatmp.add_acquisition(stimulusTimestamp) # Add the timestamp event
    csv_interface.add_to_nwbfile(datatmp) # Add csv file
    fout.export(fin, nwbfile=datatmp) # Export the new file

In [ ]:
io =  NWBHDF5IO(NWBdata_concatenate, mode='r')
nwbfile = io.read()
nwbfile

In [106]:
io.close()